In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install git+https://github.com/stan-dev/pystan2.git@master


  Cloning https://github.com/stan-dev/pystan2.git (to revision master) to /tmp/pip-req-build-yzlyp1jz
  Running command git clone --filter=blob:none --quiet https://github.com/stan-dev/pystan2.git /tmp/pip-req-build-yzlyp1jz
  Running command git checkout -b master --track origin/master
  Switched to a new branch 'master'
  Branch 'master' set up to track remote branch 'master' from 'origin'.
  Resolved https://github.com/stan-dev/pystan2.git to commit 1dd043db3c2618a9360a0f2ccbb57221634e5b08
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for pystan: filename=pystan-2.19.1.1-cp310-cp310-linux_x86_64.whl size=180503350 sha256=1a60aed9f72ba58234bf47a6717623743ea80120f7be457966876820faf25c68
  Stored in directory: /tmp/pip-ephem-wheel-cache-1nc_6ep4/wheels/13/25/0f/2d846af1d8245c80b63b0b6d1b65d24c59f60562575003bd5a
Successfully built pystan
  Attempting uninstall: pystan
    Found existing installation: pystan 3.9.0
  

# Experiment with Generated Data

In [17]:
!git clone https://github.com/zalando/expan.git


Cloning into 'expan'...
remote: Enumerating objects: 5585, done.
remote: Total 5585 (delta 0), reused 0 (delta 0), pack-reused 5585
Receiving objects: 100% (5585/5585), 1.27 MiB | 3.90 MiB/s, done.
Resolving deltas: 100% (3734/3734), done.


In [149]:
import numpy as np
import pandas as pd
import expan.core.experiment as experiment


In [150]:
np.random.seed(123)  # for reproducibility

# Parameters
num_participants = 1000
treatment_conversion_rate = 0.5  # hypothetical conversion rate in the treatment group
control_conversion_rate = 0.4    # hypothetical conversion rate in the control group

# Generate participant IDs
participants = range(num_participants)

# Randomly assign participants to either treatment or control group
group_assignment = np.random.choice(['treatment', 'control'], size=num_participants)

# Simulate outcomes based on conversion rates
outcomes = np.where(group_assignment == 'treatment', 
                    np.random.binomial(1, treatment_conversion_rate, num_participants), 
                    np.random.binomial(1, control_conversion_rate, num_participants))

# Create a DataFrame
data = pd.DataFrame({
    'participant_id': participants,
    'group': group_assignment,
    'outcome': outcomes
})


In [152]:
from expan.core.util import generate_random_data
data, metadata = generate_random_data()

In [153]:
from expan.core.experiment import Experiment
exp = Experiment(metadata=metadata)

In [154]:
data

,entity,variant,normal_same,normal_shifted,feature,normal_shifted_by_feature,treatment_start_time,normal_unequal_variance,date
0,0,A,-1.487862,-0.616148,feature that only has one data point,-1.088533,7,0.003991,2015-01-01
1,1,B,-1.125186,1.783682,has,1.167307,3,-3.565511,2015-01-01
2,2,B,0.388819,1.007539,non,-1.055948,1,6.704536,2015-01-01
3,3,A,-1.173873,-0.889252,non,-0.152459,4,1.209668,2015-01-01
4,4,A,1.112634,0.434377,has,0.175988,4,0.148207,2015-01-01
...,...,...,...,...,...,...,...,...,...
9995,9995,B,-0.028124,0.706185,non,1.270654,5,-17.878939,2016-02-21
9996,9996,B,0.792395,1.829488,non,-1.001675,4,6.321226,2016-02-21
9997,9997,B,0.994518,-1.917363,non,-0.280280,4,-0.474425,2016-02-21
9998,9998,A,-0.697330,0.359386,has,0.800866,7,-0.678204,2016-02-21


In [155]:
metadata

{'primary_KPI': 'normal_shifted',
 'source': 'simulated',
 'experiment': 'random_data_generation'}

In [156]:
from expan.core.experiment import Experiment
exp = Experiment(metadata=metadata)

In [157]:
from expan.core.statistical_test import KPI, Variants, StatisticalTest

kpi = KPI('normal_same')
variants = Variants(variant_column_name='variant', control_name='B', treatment_name='A')
test = StatisticalTest(data=data, kpi=kpi, features=[], variants=variants)

In [158]:
test.data

,entity,variant,normal_same,normal_shifted,feature,normal_shifted_by_feature,treatment_start_time,normal_unequal_variance,date
0,0,A,-1.487862,-0.616148,feature that only has one data point,-1.088533,7,0.003991,2015-01-01
1,1,B,-1.125186,1.783682,has,1.167307,3,-3.565511,2015-01-01
2,2,B,0.388819,1.007539,non,-1.055948,1,6.704536,2015-01-01
3,3,A,-1.173873,-0.889252,non,-0.152459,4,1.209668,2015-01-01
4,4,A,1.112634,0.434377,has,0.175988,4,0.148207,2015-01-01
...,...,...,...,...,...,...,...,...,...
9995,9995,B,-0.028124,0.706185,non,1.270654,5,-17.878939,2016-02-21
9996,9996,B,0.792395,1.829488,non,-1.001675,4,6.321226,2016-02-21
9997,9997,B,0.994518,-1.917363,non,-0.280280,4,-0.474425,2016-02-21
9998,9998,A,-0.697330,0.359386,has,0.800866,7,-0.678204,2016-02-21


In [159]:
exp.analyze_statistical_test(test)

{
    "result": {
        "confidence_interval": [
            {
                "percentile": 2.5,
                "value": -0.007138761863314601
            },
            {
                "percentile": 97.5,
                "value": 0.0732438406769919
            }
        ],
        "control_statistics": {
            "mean": -0.007833095215783122,
            "sample_size": 3892,
            "variance": 0.991195545472199
        },
        "delta": 0.03305253940683865,
        "p": 0.10698672122297047,
        "statistical_power": 0.36400577293301273,
        "treatment_statistics": {
            "mean": 0.025219444191055533,
            "sample_size": 6108,
            "variance": 1.0046096975922052
        }
    },
    "test": {
        "features": [],
        "kpi": {
            "name": "normal_same"
        },
        "variants": {
            "control_name": "B",
            "treatment_name": "A",
            "variant_column_name": "variant"
        }
    }
}

In [160]:
from expan.core.statistical_test import KPI, Variants, StatisticalTest

kpi = KPI('normal_same')
variants = Variants(variant_column_name='variant', control_name='B', treatment_name='A')
test = StatisticalTest(data=data, kpi=kpi, features=[], variants=variants)

In [161]:
exp.analyze_statistical_test(test, test_method='fixed_horizon', assume_normal=True)


{
    "result": {
        "confidence_interval": [
            {
                "percentile": 2.5,
                "value": -0.007138761863314601
            },
            {
                "percentile": 97.5,
                "value": 0.0732438406769919
            }
        ],
        "control_statistics": {
            "mean": -0.007833095215783122,
            "sample_size": 3892,
            "variance": 0.991195545472199
        },
        "delta": 0.03305253940683865,
        "p": 0.10698672122297047,
        "statistical_power": 0.36400577293301273,
        "treatment_statistics": {
            "mean": 0.025219444191055533,
            "sample_size": 6108,
            "variance": 1.0046096975922052
        }
    },
    "test": {
        "features": [],
        "kpi": {
            "name": "normal_same"
        },
        "variants": {
            "control_name": "B",
            "treatment_name": "A",
            "variant_column_name": "variant"
        }
    }
}

In [162]:
from expan.core.statistical_test import KPI, Variants, StatisticalTest

kpi = KPI('normal_same')
variants = Variants(variant_column_name='variant', control_name='B', treatment_name='A')
test = StatisticalTest(data=data, kpi=kpi, features=[], variants=variants)

In [163]:
exp.analyze_statistical_test(test, test_method='group_sequential', estimated_sample_size=1000)


{
    "result": {
        "confidence_interval": [
            {
                "percentile": 2.5,
                "value": -0.007134748551444851
            },
            {
                "percentile": 97.5,
                "value": 0.07323982736512216
            }
        ],
        "control_statistics": {
            "mean": -0.007833095215783122,
            "sample_size": 3892,
            "variance": 0.9909408703577406
        },
        "delta": 0.03305253940683865,
        "p": 0.10695169535461431,
        "statistical_power": 0.36400577293301273,
        "stop": false,
        "treatment_statistics": {
            "mean": 0.025219444191055533,
            "sample_size": 6108,
            "variance": 1.0044452231819903
        }
    },
    "test": {
        "features": [],
        "kpi": {
            "name": "normal_same"
        },
        "variants": {
            "control_name": "B",
            "treatment_name": "A",
            "variant_column_name": "variant"
       

# Experiment with Kaggle fast-food data https://www.kaggle.com/datasets/chebotinaa/fast-food-marketing-campaign-ab-test/versions/1

In [164]:
data2=pd.read_csv("/kaggle/input/fast-food-marketing-campaign-ab-test/WA_Marketing-Campaign.csv")

In [165]:
data2['MarketSize'].unique()

array(['Medium', 'Small', 'Large'], dtype=object)

In [166]:
data2['entity'] = data2.index


In [171]:
data2.groupby('MarketSize').mean('SalesInThousands')

,MarketID,LocationID,AgeOfStore,Promotion,week,SalesInThousands,entity
MarketSize,,,,,,,
Large,6.333333,544.357143,7.142857,1.952381,2.5,70.116726,304.261905
Medium,5.862500,493.550000,8.787500,2.062500,2.5,43.985344,283.450000
Small,3.200000,224.400000,10.800000,2.066667,2.5,57.409333,134.300000


# Large vs Medium Comparison

In [167]:
data=data2[data2['MarketSize'].isin(['Large', 'Medium'])]

In [168]:
from scipy import stats

# Assuming `data` is your DataFrame and it has been filtered for 'Large' and 'Medium' market sizes as per your request
# And assuming 'SalesInThousands' is the column of interest

# Separating the data into two groups: Large and Medium
large_sales = data2[data2['MarketSize'] == 'Large']['SalesInThousands']
medium_sales = data2[data2['MarketSize'] == 'Medium']['SalesInThousands']

# Performing normality tests for both groups
normality_test_large = stats.shapiro(large_sales)
normality_test_medium = stats.shapiro(medium_sales)

normality_test_large, normality_test_medium


(ShapiroResult(statistic=0.9209785461425781, pvalue=6.401694463420426e-08),
 ShapiroResult(statistic=0.9926648139953613, pvalue=0.11779474467039108))

For the 'Large' market size group, the sales data does not appear to be normally distributed based on the Shapiro-Wilk test. In contrast, the 'Medium' market size group's sales data does not significantly deviate from normality, and thus, could be considered normally distributed based on this test.

In [169]:
data

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands,entity
0,1,Medium,1,4,3,1,33.73,0
1,1,Medium,1,4,3,2,35.67,1
2,1,Medium,1,4,3,3,29.03,2
3,1,Medium,1,4,3,4,39.25,3
4,1,Medium,2,5,2,1,27.81,4
...,...,...,...,...,...,...,...,...
543,10,Large,919,2,1,4,64.34,543
544,10,Large,920,14,2,1,50.20,544
545,10,Large,920,14,2,2,45.75,545
546,10,Large,920,14,2,3,44.29,546


In [130]:
from expan.core.statistical_test import KPI, Variants, StatisticalTest

kpi = KPI('SalesInThousands')
variants = Variants(variant_column_name='MarketSize', control_name='Large', treatment_name='Medium')
test = StatisticalTest(data=data, kpi=kpi, features=[], variants=variants)

In [126]:
exp.analyze_statistical_test(test, test_method='fixed_horizon', assume_normal=False)


{
    "result": {
        "confidence_interval": [
            {
                "percentile": 2.5,
                "value": -28.869150148809528
            },
            {
                "percentile": 97.5,
                "value": -23.368075930059526
            }
        ],
        "control_statistics": {
            "mean": 70.1167261904762,
            "sample_size": 168,
            "variance": 289.0451994012188
        },
        "delta": -26.131382440476195,
        "p": 8.284548482156341e-76,
        "statistical_power": 1.0,
        "treatment_statistics": {
            "mean": 43.985343750000006,
            "sample_size": 320,
            "variance": 81.67142925683592
        }
    },
    "test": {
        "features": [],
        "kpi": {
            "name": "SalesInThousands"
        },
        "variants": {
            "control_name": "Large",
            "treatment_name": "Medium",
            "variant_column_name": "MarketSize"
        }
    }
}

Statistical Significance: The p-value is extremely low (8.28e-76), far below any conventional significance level (e.g., 0.05), indicating a statistically significant difference between the control and treatment groups.

Statistical Power: The statistical power of the test is 1.0, suggesting that the test had a very high probability of correctly rejecting the null hypothesis (i.e., detecting a true effect when there is one).

Control Group Statistics:

Mean sales in thousands: 70.117
Sample size: 168
Variance: 289.045
Treatment Group Statistics:

Mean sales in thousands: 43.985
Sample size: 320
Variance: 81.671




# Medium vs Small Comparison

In [138]:
from scipy import stats

# Assuming `data` is your DataFrame and it has been filtered for 'Large' and 'Medium' market sizes as per your request
# And assuming 'SalesInThousands' is the column of interest

# Separating the data into two groups: Large and Medium
small_sales = data2[data2['MarketSize'] == 'Small']['SalesInThousands']

# Performing normality tests for both groups
normality_test_small = stats.shapiro(small_sales)

normality_test_small

ShapiroResult(statistic=0.9777514934539795, pvalue=0.3409940004348755)

**Small size market sales data could be considered normally distributed based on Shapiro Wilk test.**

In [140]:
data=data2[data2['MarketSize'].isin(['Small', 'Medium'])]

In [141]:
from expan.core.statistical_test import KPI, Variants, StatisticalTest

kpi = KPI('SalesInThousands')
variants = Variants(variant_column_name='MarketSize', control_name='Medium', treatment_name='Small')
test = StatisticalTest(data=data, kpi=kpi, features=[], variants=variants)

In [142]:
exp.analyze_statistical_test(test, test_method='fixed_horizon', assume_normal=True)


{
    "result": {
        "confidence_interval": [
            {
                "percentile": 2.5,
                "value": 11.012365776249524
            },
            {
                "percentile": 97.5,
                "value": 15.835613390417095
            }
        ],
        "control_statistics": {
            "mean": 43.985343750000006,
            "sample_size": 320,
            "variance": 81.92745254604232
        },
        "delta": 13.42398958333331,
        "p": 2.102173527805208e-24,
        "statistical_power": 1.0,
        "treatment_statistics": {
            "mean": 57.409333333333315,
            "sample_size": 60,
            "variance": 43.998030056497186
        }
    },
    "test": {
        "features": [],
        "kpi": {
            "name": "SalesInThousands"
        },
        "variants": {
            "control_name": "Medium",
            "treatment_name": "Small",
            "variant_column_name": "MarketSize"
        }
    }
}


Statistical Significance: The p-value is extremely low (2.10e-24), indicating that the difference in sales between the "Small" market size (treatment) and the "Medium" market size (control) is statistically significant.

Statistical Power: The test has a statistical power of 1.0, which means there is a very high likelihood that the test correctly identified a true effect (i.e., a significant difference in sales between the two groups).


Control Group Statistics:
Mean sales in thousands: 43.985
Sample size: 320
Variance: 81.927

Treatment Group Statistics:
Mean sales in thousands: 57.409
Sample size: 60
Variance: 43.998


# Large  vs Small Comparison

In [143]:
data=data2[data2['MarketSize'].isin(['Small', 'Large'])]

In [147]:
from expan.core.statistical_test import KPI, Variants, StatisticalTest

kpi = KPI('SalesInThousands')
variants = Variants(variant_column_name='MarketSize', control_name='Large', treatment_name='Small')
test = StatisticalTest(data=data, kpi=kpi, features=[], variants=variants)

In [148]:
exp.analyze_statistical_test(test, test_method='fixed_horizon', assume_normal=False)


{
    "result": {
        "confidence_interval": [
            {
                "percentile": 2.5,
                "value": -15.782952976190483
            },
            {
                "percentile": 97.5,
                "value": -9.576246130952393
            }
        ],
        "control_statistics": {
            "mean": 70.1167261904762,
            "sample_size": 168,
            "variance": 289.0451994012188
        },
        "delta": -12.707392857142885,
        "p": 5.201529357184487e-08,
        "statistical_power": 0.9998807819110238,
        "treatment_statistics": {
            "mean": 57.409333333333315,
            "sample_size": 60,
            "variance": 43.264729555555554
        }
    },
    "test": {
        "features": [],
        "kpi": {
            "name": "SalesInThousands"
        },
        "variants": {
            "control_name": "Large",
            "treatment_name": "Small",
            "variant_column_name": "MarketSize"
        }
    }
}

Statistical Significance: The p-value is extremely low (5.20e-08), far below the conventional threshold of 0.05. This indicates a statistically significant difference in sales between the "Small" market size (treatment) and the "Large" market size (control).

Statistical Power: The test's statistical power is nearly 1.0 (0.9998807819110238), meaning it has a very high likelihood of correctly rejecting the null hypothesis when it is false (i.e., detecting a true effect when it exists).

Confidence Interval: The 95% confidence interval for the effect size ranges from -15.783 to -9.576 thousand sales. This interval suggests a high level of certainty that the true effect size (difference in sales) is negative and lies within this range.

Control Group Statistics:

Mean sales in thousands: 70.117
Sample size: 168 participants
Variance: 289.045
Treatment Group Statistics:

Mean sales in thousands: 57.409
Sample size: 60 participants
Variance: 43.265
Delta (Effect Size): The difference in average sales between the control and treatment groups is -12.707 thousand, indicating that the treatment group (Small market size) has lower sales compared to the control group (Large market size).



Based on the pairwise A/B test results, it can be conclusively determined that the sales performance, as measured in thousands, follows a distinct order across different market sizes: "Large" outperforms "Small", which in turn outperforms "Medium". 
Large> Small> Medium 